In [30]:
baseline_test_prediction = 'E:/projects/SolveItEasy/exps_modal/bfloat16_lora_r16_a32/results/test_predictions.txt'
finetuned_test_prediction = 'E:/projects/SolveItEasy/exps_modal/bfloat16_lora_r16_a32/results/test_predictions_ep4.txt'

In [31]:
import re

def parse_predictions(file_path):
    examples = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        entries = content.split("id: ")[1:]  # Split by id but keep entries
        
        for entry in entries:
            id_match = re.search(r"^(\d+)", entry)
            correct_match = re.search(r"correct: (True|False)", entry)
            
            if id_match and correct_match:
                example_id = int(id_match.group(1))
                correct = correct_match.group(1) == "True"
                examples[example_id] = correct
    
    return examples

def find_improvements(baseline_file, finetuned_file):
    baseline_results = parse_predictions(baseline_file)
    finetuned_results = parse_predictions(finetuned_file)
    
    improved_examples = [
        example_id for example_id in baseline_results
        if not baseline_results[example_id] and finetuned_results.get(example_id, False)
    ]
    
    return baseline_results, finetuned_results, improved_examples




In [32]:
# Example usage:
# baseline_results, finetuned_results, improved_examples = find_improvements(baseline_test_prediction, finetuned_test_prediction)
baseline_results, finetuned_results, improved_examples = find_improvements(finetuned_test_prediction, baseline_test_prediction)
print(improved_examples)

[2, 4, 5, 12, 16, 17, 19, 21, 27, 28, 31, 33, 35, 36, 37, 38, 39, 43, 50, 53, 63, 64, 68, 70, 72, 73, 80, 81, 82, 92, 97, 99, 101, 103, 108, 109, 113, 118, 120, 121, 122, 125, 126, 128, 130, 137, 138, 140, 141, 142, 144, 155, 164, 165, 167, 170, 172, 173, 178, 181, 183, 186, 189, 192, 194, 196, 197, 204, 206, 207, 212, 213, 216, 219, 221, 227, 235, 242, 244, 249, 250, 255, 257, 263, 266, 271, 273, 283, 286, 292, 295, 296, 308, 315, 322, 324, 326, 333, 336, 338, 339, 342, 346, 348, 349, 354, 357, 358, 360, 362, 364, 367, 370, 376, 377, 382, 385, 386, 387, 401, 404, 405, 408, 410, 415, 421, 425, 430, 436, 437, 441, 442, 445, 447, 455, 456, 459, 460, 465, 467, 473, 475, 477, 496, 497, 499, 500, 505, 519, 521, 522, 526, 529, 533, 534, 536, 541, 549, 552, 554, 559, 563, 564, 585, 589, 609, 622, 628, 632, 633, 635, 640, 643, 650, 651, 653, 656, 657, 660, 661, 662, 682, 683, 684, 688, 690, 698, 700, 701, 704, 706, 709, 712, 713, 716, 722, 723, 724, 727, 728, 729, 738, 740, 742, 745, 747, 750,

In [24]:
len(improved_examples)

157

In [ ]:
import re
import pandas as pd
def parse_predictions_baseline(file_path):
    examples = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        entries = content.split("id: ")[1:]  # Split by id but keep entries
        
        for entry in entries:
            id_match = re.search(r"^(\d+)", entry)
            problem_match = re.search(r"Here is the problem:\n(.*?)\n\nNow", entry, re.DOTALL)
            solution_match = re.search(r"Solution:\s*(Solution:.*?)?\n------------------------", entry, re.DOTALL)
            correct_match = re.search(r"correct: (True|False)", entry)
            
            if id_match and problem_match and solution_match and correct_match:
                example_id = int(id_match.group(1))
                try:
                    problem = problem_match.group(1).strip()
                except:
                    problem = "error"

                try:
                    solution = solution_match.group(1).strip()
                except:
                    solution = "error"

                # solution = solution_match.group(1).strip()
                correct = correct_match.group(1) == "True"
                examples[example_id] = (problem, solution, correct)

                print(examples[example_id])
    
    return examples


def parse_predictions(file_path):
    examples = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        entries = content.split("id: ")[1:]  # Split by id but keep entries
        
        for entry in entries:
            id_match = re.search(r"^(\d+)", entry)
            problem_match = re.search(r"Here is the problem:\n(.*?)\n\nNow", entry, re.DOTALL)
            solution_match = re.search(r"Solution:\s*(Solution:.*?)\n------------------------", entry, re.DOTALL)
            correct_match = re.search(r"correct: (True|False)", entry)
            
            if id_match and problem_match and solution_match and correct_match:
                example_id = int(id_match.group(1))
                try:
                    problem = problem_match.group(1).strip()
                except:
                    problem = "error"

                try:
                    solution = solution_match.group(1).strip()
                except:
                    solution = "error"

                # solution = solution_match.group(1).strip()
                correct = correct_match.group(1) == "True"
                examples[example_id] = (problem, solution, correct)

                print(examples[example_id])
    
    return examples


def build_comparison_df(baseline_file, finetuned_file):
    baseline_results = parse_predictions(baseline_file)
    print("baseline_results:", baseline_results)
    
    finetuned_results = parse_predictions(finetuned_file)
    print("finetuned_results:", finetuned_results)
    
    common_ids = set(baseline_results.keys()) & set(finetuned_results.keys())
    
    data = [
        [
            example_id, 
            baseline_results[example_id][0], 
            baseline_results[example_id][1], 
            finetuned_results[example_id][1], 
            baseline_results[example_id][2], 
            finetuned_results[example_id][2]
        ]
        for example_id in common_ids
    ]
    
    df = pd.DataFrame(data, columns=["Test Sample ID", "Problem", "Baseline Solution", "Finetuned Solution", "Baseline Correct", "Finetuned Correct"])
    return baseline_results, finetuned_results, df
    # return df

# def build_comparison_df(baseline_file, finetuned_file):
#     baseline_results = parse_predictions(baseline_file)
#     finetuned_results = parse_predictions(finetuned_file)
    
#     data = []
#     for example_id in baseline_results:
#         if example_id in finetuned_results:
#             problem, baseline_solution, baseline_correct = baseline_results[example_id]
#             _, finetuned_solution, finetuned_correct = finetuned_results[example_id]
#             data.append([example_id, problem, baseline_solution, finetuned_solution, baseline_correct, finetuned_correct])
    
#     df = pd.DataFrame(data, columns=["Test Sample ID", "Problem", "Baseline Solution", "Finetuned Solution", "Baseline Correct", "Finetuned Correct"])
#     return baseline_results, finetuned_results, df


In [60]:
# Example usage:
baseline_results, finetuned_results, df_comparison = build_comparison_df(baseline_test_prediction, finetuned_test_prediction)
print(df_comparison)


baseline_results: {}
('Vịt của Janet đẻ được 16 quả trứng mỗi ngày. Cô ấy ăn 3 quả vào bữa sáng và nướng bánh muffin cho bạn bè mỗi ngày với 4 quả. Cô ấy bán số trứng còn lại tại chợ nông sản hàng ngày với giá $2 mỗi quả trứng vịt tươi. Cô ấy kiếm được bao nhiêu đô la mỗi ngày tại chợ nông sản?', 'Solution:\nReason:\nCô ấy có 16 quả trứng - 3 quả trứng = 13 quả trứng còn lại sau khi ăn sáng.\nCô ấy có 13 quả trứng - 4 quả trứng = 9 quả trứng còn lại sau khi làm bánh.\nCô ấy kiếm được 9 quả trứng * $2/quả trứng = $18 mỗi ngày tại chợ nông sản.\nAnswer:\n18', True)
('Một chiếc áo choàng cần 2 cuộn sợi màu xanh và một nửa số đó là sợi màu trắng. Tổng cộng cần bao nhiêu cuộn sợi?', 'Solution:\nReason:\nMột chiếc áo choàng cần 1 cuộn sợi màu trắng vì 2/2 = 1.\nVì vậy, tổng cộng cần 3 cuộn sợi vì 2 + 1 = 3.\nAnswer:\n3', True)
('Josh quyết định thử sức với việc mua bán nhà. Anh ấy mua một căn nhà với giá $80,000 và sau đó chi thêm $50,000 để sửa chữa. Việc sửa chữa này đã làm tăng giá trị củ

In [56]:
len(baseline_results), len(finetuned_results)

(0, 1318)

In [54]:
df_comparison

,Test Sample ID,Problem,Baseline Solution,Finetuned Solution,Baseline Correct,Finetuned Correct


In [77]:
def parse_predictions_baseline(file_path):
    examples = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        entries = content.split("id: ")[1:]  # Split by id but keep entries
        
        for entry in entries:
            id_match = re.search(r"^(\d+)", entry)
            problem_match = re.search(r"Here is the problem:\n(.*?)\n\nNow", entry, re.DOTALL)
            solution_match = re.search(r"Solution:\s*(Solution:.*?)?\n------------------------", entry, re.DOTALL)
            correct_match = re.search(r"correct: (True|False)", entry)
            
            if id_match and problem_match and solution_match and correct_match:
                example_id = int(id_match.group(1))
                try:
                    problem = problem_match.group(1).strip()
                except:
                    problem = "error"
                    # continue

                try:
                    solution = solution_match.group(1).strip()
                except:
                    solution = "error"
                    # continue

                # solution = solution_match.group(1).strip()
                correct = correct_match.group(1) == "True"
                examples[example_id] = (problem, solution, correct)

                print(examples[example_id])
    
    return examples



In [78]:
baseline_results = parse_predictions_baseline(baseline_test_prediction)
baseline_results

('Russell làm việc tại một cửa hàng thú cưng và đang phân phát rơm cho các loài gặm nhấm. Những con chuột được giữ trong 3 lồng, mỗi lồng chứa số chuột bằng nhau và mỗi con chuột được cho 6 miếng rơm. Có 10 lồng chuột nhắt được giữ riêng biệt và mỗi con chuột được cho 5 miếng rơm. Có một chuồng thỏ nơi 20 miếng rơm được phân phát cho các con thỏ. Không có rơm nào được sử dụng ở bất kỳ nơi nào khác trong cửa hàng. Nếu đã phân phát 160 miếng rơm cho các loài gặm nhấm nhỏ, thì có bao nhiêu con chuột trong mỗi lồng?', 'error', True)


{186: ('Russell làm việc tại một cửa hàng thú cưng và đang phân phát rơm cho các loài gặm nhấm. Những con chuột được giữ trong 3 lồng, mỗi lồng chứa số chuột bằng nhau và mỗi con chuột được cho 6 miếng rơm. Có 10 lồng chuột nhắt được giữ riêng biệt và mỗi con chuột được cho 5 miếng rơm. Có một chuồng thỏ nơi 20 miếng rơm được phân phát cho các con thỏ. Không có rơm nào được sử dụng ở bất kỳ nơi nào khác trong cửa hàng. Nếu đã phân phát 160 miếng rơm cho các loài gặm nhấm nhỏ, thì có bao nhiêu con chuột trong mỗi lồng?',
  'error',
  True)}

In [79]:
len(baseline_results)

1